In [37]:
import sqlite3
import random
import re

In [38]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    city TEXT
)
''')

names = ["Alice", "Bob", "Charlie", "Dante", "Eva"]
cities = ["New York", "Paris", "London", "Tokyo", "Berlin"]

for _ in range(50):
    cursor.execute(
        "INSERT INTO users (name, age, city) VALUES (?, ?, ?)",
        (random.choice(names), random.randint(10, 60), random.choice(cities))
    )
conn.commit()

In [39]:
def nlq_to_sql(text):
    text = text.lower()
    conditions = []

    # Age filters
    if match := re.search(r"older than (\d+)", text):
        conditions.append(f"age > {match.group(1)}")
    if match := re.search(r"younger than (\d+)", text):
        conditions.append(f"age < {match.group(1)}")
    if match := re.search(r"between (\d+) and (\d+)", text):
        conditions.append(f"age BETWEEN {match.group(1)} AND {match.group(2)}")

    # City filter
    if match := re.search(r"from (\w+)", text):
        city = match.group(1).capitalize()
        conditions.append(f"city = '{city}'")

    # Select columns
    if "average age" in text:
        select = "AVG(age)"
    elif "names" in text:
        select = "name"
    else:
        select = "*"

    # Build query
    sql = f"SELECT {select} FROM users"
    if conditions:
        sql += " WHERE " + " AND ".join(conditions)
    return sql

In [40]:
queries = [
    "Show all users",
    "List names",
    "Users older than 30",
    "Users younger than 25 from Tokyo",
    "Users between 20 and 30",
    "Average age of users",
    "Average age of users from Paris between 20 and 40"
]

for q in queries:
    sql = nlq_to_sql(q)
    print(f"\nNLQ: {q}\nSQL: {sql}")
    cursor.execute(sql)
    result = cursor.fetchall()
    print("Result:", result)

conn.close()


NLQ: Show all users
SQL: SELECT * FROM users
Result: [(1, 'Alice', 55, 'London'), (2, 'Charlie', 22, 'Tokyo'), (3, 'Bob', 15, 'London'), (4, 'Bob', 11, 'New York'), (5, 'Bob', 60, 'New York'), (6, 'Alice', 32, 'Berlin'), (7, 'Dante', 24, 'London'), (8, 'Eva', 30, 'New York'), (9, 'Bob', 24, 'New York'), (10, 'Bob', 47, 'Berlin'), (11, 'Alice', 41, 'London'), (12, 'Eva', 57, 'Paris'), (13, 'Dante', 56, 'London'), (14, 'Eva', 53, 'Paris'), (15, 'Charlie', 47, 'Paris'), (16, 'Dante', 34, 'London'), (17, 'Eva', 20, 'Paris'), (18, 'Bob', 10, 'Tokyo'), (19, 'Eva', 60, 'London'), (20, 'Dante', 32, 'Paris'), (21, 'Dante', 24, 'Paris'), (22, 'Dante', 28, 'Paris'), (23, 'Dante', 22, 'Berlin'), (24, 'Dante', 19, 'Paris'), (25, 'Charlie', 29, 'Tokyo'), (26, 'Eva', 12, 'Paris'), (27, 'Alice', 21, 'Berlin'), (28, 'Alice', 24, 'New York'), (29, 'Dante', 47, 'New York'), (30, 'Dante', 23, 'Paris'), (31, 'Alice', 33, 'Tokyo'), (32, 'Alice', 27, 'Paris'), (33, 'Bob', 12, 'Berlin'), (34, 'Alice', 33, 'N